## 각 디렉토리(클래스)별로 train / val 으로 나누어 학습한 코드
------------------------------------------------------------
### kfashion 데이터셋의 디렉토리 별로 반복문을 사용해서 train/ val로 나누고 이 데이터셋을 학습시켰습니다. 
### 생성된 가중치 파일을 다시 불러와 train/val로 나눈 다음 디렉토리를 학습시켰습니다.
### 위 두개의 과정을 계속 반복해, 각 클래스별로 학습시켰습니다.
### kfashion 데이터셋을 로드하는 과정에서 데이터가 깨져서 발생한 에러때문에 학습이 되지 않습니다...😪

## 환경 설정

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install kfashion
!rm -rf Kfashion
!git clone https://github.com/sanso62/Kfashion.git
%cd Kfashion

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 49.7 MB 327 kB/s 
     |████████████████████████████████| 190 kB 27.6 MB/s 
Cloning into 'Kfashion'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 203 (delta 74), reused 142 (delta 53), pack-reused 0
Receiving objects: 100% (203/203), 2.48 MiB | 27.94 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/Kfashion/mmdetection/Kfashion
Cloning into 'mmdetection'...
remote: Enumerating objects: 32889, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (

In [5]:
from mmcv import collect_env
collect_env()

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


{'sys.platform': 'linux',
 'Python': '3.8.16 (default, Dec  7 2022, 01:12:13) [GCC 7.5.0]',
 'CUDA available': True,
 'GPU 0': 'Tesla T4',
 'CUDA_HOME': '/usr/local/cuda',
 'NVCC': 'Cuda compilation tools, release 11.2, V11.2.152',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86

In [6]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.26.0
11.1
GCC 7.3


## Fine Tunning: Kfashion Dataset

Data Preprocessing

In [7]:
# load dataset
# !wget https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
# !unzip -q balloon_dataset.zip
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot, set_random_seed
from mmdet.models import build_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import datetime

In [9]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

import json
import zipfile
import os
import shutil
import re
import random
import sys

In [10]:
try:
    shutil.rmtree("/content/Kfashion/mmdetection/datasetzip")
except:
    pass

In [11]:
!rm -rf datasetzip
!mkdir datasetzip
!unzip -q /content/drive/MyDrive/Kfashion_dataset.zip -d /content/Kfashion/mmdetection/datasetzip

In [12]:
# coco dataset으로 프리 트레이닝된 mask_rcnn config 가져오기
from mmcv import Config
cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py')


In [13]:
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O ../log/latest.pth

--2022-12-16 14:14:46--  https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.82.219, 163.181.82.218, 163.181.82.215, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.82.219|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 177867103 (170M), 177867102 (170M) remaining [application/octet-stream]
Saving to: ‘../log/latest.pth’

../log/latest.pth   100%[===================>] 169.63M  12.8MB/s    in 16s     

2022-12-16 14:15:06 (10.5 MB/s) - ‘../log/latest.pth’ saved [177867103/177867103]



In [14]:
categories = [{"id": 0, "name": "coat"},
              {"id": 1, "name": "jacket"},
              {"id": 2, "name": "jumper"},
              {"id": 3, "name": "cardigan"},
              {"id": 4, "name": "blouse"},
              {"id": 5, "name": "t-shirt"},
              {"id": 6, "name": "sweater"},
              {"id": 7, "name": "shirt"},
              {"id": 8, "name": "onepiece(dress)"},
              {"id": 9, "name": "jumpsuite"},
              {"id": 10, "name": "pants"},
              {"id": 11, "name": "skirt"}]

In [16]:
for (_, _, files) in os.walk("/content/Kfashion/mmdetection/datasetzip"):
    for i,f in enumerate(files):
        try:
            print("**")
            shutil.rmtree("/content/Kfashion/mmdetection/kfashion")
        except:
            pass
        os.mkdir("/content/Kfashion/mmdetection/kfashion/")
        zipfile.ZipFile("/content/Kfashion/mmdetection/datasetzip/"+f).extractall("/content/Kfashion/mmdetection/kfashion/"+f)
        # shutil.unpack_archive("/content/Kfashion/mmdetection/datasetzip/"+f, "/content/Kfashion/mmdetection/kfashion/"+f, "zip")
        os.mkdir("/content/Kfashion/mmdetection/kfashion/"+f+"/train")
        os.mkdir("/content/Kfashion/mmdetection/kfashion/"+f+"/val")
        print(dir)
        train_img_id_list={}
        val_img_id_list={}
        train_images=[]
        val_images=[]
        train_annotations=[]
        val_annotations=[]
        with open("/content/Kfashion/mmdetection/kfashion/"+f+"/annotations/instances_default.json", "r") as file:
            label=json.load(file)
            random.shuffle(label["images"])
            len_images=len(label["images"])
            for img in label["images"][:int(len_images*0.9)]:
                file_name=re.sub("/.*/","",img["file_name"])
                train_img_id_list[file_name]=img["id"]
                img["file_name"]=file_name
                train_images.append(img)   

            for img in label["images"][int(len_images*0.9):]:
                file_name=re.sub("/.*/","",img["file_name"])
                val_img_id_list[file_name]=img["id"]
                img["file_name"]=file_name
                val_images.append(img)  

            category=label["categories"][0]["name"]
            for c in categories:
              if c["name"]==category:
                category_id=c["id"]
                break

            for ann in label["annotations"]:
                ann["category_id"]=category_id
                if ann["image_id"] in train_img_id_list.values():
                    train_annotations.append(ann)
                elif ann["image_id"] in val_img_id_list.values():
                    val_annotations.append(ann)

                


        jsondata=dict([("categories", categories), ("images", train_images), ("annotations", train_annotations) ])
        with open("/content/Kfashion/mmdetection/kfashion/"+f+"/train/annotation_kfashion.json", "w") as tf:
            json.dump(jsondata, tf)
        
        jsondata=dict([("categories", categories), ("images", val_images), ("annotations", val_annotations) ])
        with open(f"/content/Kfashion/mmdetection/kfashion/{f}/val/annotation_kfashion.json", "w") as vf:
            json.dump(jsondata, vf)

        for img_name in train_img_id_list.keys():        
            shutil.move("/content/Kfashion/mmdetection/kfashion/"+f+"/images/"+img_name, "/content/Kfashion/mmdetection/kfashion/"+f+"/train/"+img_name)
        for img_name in val_img_id_list.keys():        
            shutil.move("/content/Kfashion/mmdetection/kfashion/"+f+"/images/"+img_name, "/content/Kfashion/mmdetection/kfashion/"+f+"/val/"+img_name)
    
    print("data done")
    # Modify dataset type and path

    cfg.dataset_type = 'COCODataset'


    cfg.data.test.ann_file = "/content/Kfashion/mmdetection/kfashion/"+f+"/val/annotation_kfashion.json"
    cfg.data.test.img_prefix = "/content/Kfashion/mmdetection/kfashion/"+f+"/val/"
    cfg.data.test.classes = ("coat", "jacket", "jumper", "cardigan", "blouse", "t-shirt","sweater", "shirt", "onepiece(dress)", "jumpsuite", "pants", "skirt")


    cfg.data.train.ann_file = "/content/Kfashion/mmdetection/kfashion/"+f+"/train/annotation_kfashion.json"
    cfg.data.train.img_prefix = "/content/Kfashion/mmdetection/kfashion/"+f+"/train/"
    cfg.data.train.classes = ("coat", "jacket", "jumper", "cardigan", "blouse", "t-shirt","sweater", "shirt", "onepiece(dress)", "jumpsuite", "pants", "skirt")


    cfg.data.val.ann_file = "/content/Kfashion/mmdetection/kfashion/"+f+"/val/annotation_kfashion.json"
    cfg.data.val.img_prefix = "/content/Kfashion/mmdetection/kfashion/"+f+"/val/"
    cfg.data.val.classes = ("coat", "jacket", "jumper", "cardigan", "blouse", "t-shirt","sweater", "shirt", "onepiece(dress)", "jumpsuite", "pants", "skirt")


    # # modify num classes of the model in box head
    # cfg.model.roi_head.bbox_head.num_classes = 80
    # # modify num classes of the model in box head and mask head
    cfg.model.roi_head.bbox_head.num_classes = 12
    cfg.model.roi_head.mask_head.num_classes = 12
    # If we need to finetune a model based on a pre-trained detector, we need to
    # use load_from to set the path of log.
    cfg.load_from = '../log/latest.pth'

    # Set up working dir to save files and logs.
    cfg.work_dir = '../log/'

    # The original learning rate (LR) is set for 8-GPU training.
    # We divide it by 8 since we only use one GPU.
    cfg.optimizer.lr = 0.02 / 8
    cfg.lr_config.warmup = None
    cfg.log_config.interval = 10

    # Change the evaluation metric since we use customized dataset.
    # cfg.evaluation.metric = 'mAP'/
    # We can set the evaluation interval to reduce the evaluation times
    cfg.evaluation.interval = 12
    # We can set the checkpoint saving interval to reduce the storage cost
    cfg.checkpoint_config.interval = 12

    cfg.seed = 0
    set_random_seed(0, deterministic=False)
    cfg.device = 'cuda'
    cfg.gpu_ids = range(1)

    cfg.log_config.hooks = [
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')]
    print(f'Config:\n{cfg.pretty_text}')

    print("Config done")


    datasets = [build_dataset(cfg.data.train)]
    model = build_detector(cfg.model)
    model.CLASSES = datasets[0].CLASSES
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    # add timestamp
    train_detector(model, datasets, cfg, distributed=False, validate=True, timestamp=datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
**
<built-in function dir>
data done
Config:
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        i

2022-12-16 14:15:50,241 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-16 14:15:50,252 - mmdet - INFO - load checkpoint from local path: ../log/latest.pth


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


UnpicklingError: ignored

## 로그 시각화

In [ ]:

%load_ext tensorboard
%tensorboard --logdir ./tutorial_exps

# Detection 결과

In [ ]:
img = mmcv.imread('./balloon/sample.jpg') # 테스트할 샘플이미지의 경로명 입력

model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)